<a href="https://colab.research.google.com/github/Cassio-Tieres/automacao_de_emails/blob/main/envia_emails_migracao_AW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto

O objetivo do projeto é automatizar o disparo de notificações quanto a migração do servidor dos sites da AW.

# Funcionamento

Os sites que tem o IP apontado para a hostgator receberão um e-mail informando que ele será migrado para outro servidor.

Os sites que já estiverem com o IP apontado para infonet não precisarão ser notificados.

Já os sites que estão com o IP diferente do da hostgator ou da infonet não receberão, também, nenhum e-mail, porém, uma mensagem será disparada para Wallace (TI) a fim de confirmar que o cliente não faz parte da Alfama.

# Libs e funções utilizadas

## Libs
Apenas três bibliotecas foram utilizadas para que o algoritmo funcionasse: smtplib, socket e email.message.

Todas as três libs são built-in: ou seja, já vem por padrão na linguagem Python e não precisam ser instaladas.

* Socket: recupera o IP do site;
* SMTPLIB: configura servidor SMTP;
* email.message: configura envio da mensagem

## Funções

Uma função foi criada apenas para configurar o disparo de e-mail: enviar_email().

Esta função possue 4 parâmetros essenciais para funcionar e elas são autoexplicativas:

* <b>Assunto</b>: Descreve o assunto do email;
* <b>De</b>: de quem partirá o e-mail;
* <b>Senha</b>: senha do app (lembrando das configurações essenciais de e-mail do google);
* <b>Para</b>: pessoa que irá receber a mensagem;
* <b>Mensagem</b>: mensagem enviada.

## Código

Para que o email fizesse relação com o site no momento do envio, foi criado um dicionário que contém chave e valor.

As chaves do dicionário são os e-mails para quais a mensagem sobre a migração será enviada. O valor é o domínio do site.


````python
sites = {
  'email_cliente': 'site.com.br',
}
````

Após a configuração do dicionário, é feito um loop para iterar sobre a chave e o valor desta.

Uma condicional é criada para verificar se o IP do site está apontado para o da hostgator ou o da infonet e caso não esteja em nenhum, cairá no "else", enviando o e-mail para o TI.

### Disparo

O disparo do e-mail é feito através da tratativa de exceções, usando try-catch.
Caso o envio seja bem executado, então a função enviar_email é chamada com as especificações em seus parâmetros, se não, o console retornará um erro.

````python
try:
      envia_email(f'Migração de servidor do site - {site}', 'email_de_disparo', 'senha_de_app', email, msg_migracao)
      envia_email(f'Confirmação de envio - {site}', 'email_de_disparo', 'senha_de_app', 'email_recebimento', msg_enviou)
except Exception as e:
      print(f'Erro ao tentar enviar e-mail. Erro: {e}')
````


In [ ]:
#importação de libs necessárias
import socket as skt
import smtplib as smtp
import email.message as em

In [ ]:
def envia_email(assunto, de, senha, para, mensagem):
  email_config = em.EmailMessage()
  email_config['Subject'] = assunto
  email_config['From'] = de
  email_config['To'] = para
  password = senha
  email_config.add_header('Content-Type', 'text/html')
  email_config.set_payload(mensagem)

  s = smtp.SMTP('smtp.gmail.com: 587')
  s.starttls()

  s.login(email_config['From'], password)
  s.sendmail(email_config['From'], [email_config['To']], email_config.as_string().encode('utf-8'))

  print('Email enviado')

In [ ]:
print(' ---------------------------------')
print('|       enviar e-mails           |')
print(' ---------------------------------')

# dicionário de clientes. O índice deve conter o e-mail e o valor o site
sites = {
  'email@email.com':'site.com.br'
}

for email, site in sites.items():
  print('-----------------------------------------------------------------------------------------------------------')

  if skt.gethostbyname(site) == 'IP1':
    msg_migracao = f'''
      Olá!
      <br>
      <br>
      Aqui quem fala é Cássio.
      <br>
      <br>
      Este e-mail está sendo enviado pois estamos passando por um processo de migração de servidores. <br>
      Identificamos que o site {site} é um dos sites que precisará passar pela migração.
      <br>
      <br>
      Por isso, solicitamos o seu suporte nos fornecendo o acesso ao servidor DNS do site ou auxiliando nos apontamentos.
      <br>
      <br>
      Temos até o da 20/06 para aplicarmos a migração do seu site.
      <br>
      <br>
      Você consegue nos ajudar com isso?
      <br>
      <br>
      Att, <br>
      Cássio & Alfama Web
    '''
    msg_enviou = f'''
    Esta mensagem está sendo enviada para confirmar o envio do e-mail para {email} sobre a migração de servidor para o site {site} <br>
    Att,<br>
    Cássio Tieres - Desenvolvedor da AW.
    '''
    print(f'O site {site} está hospedado na hostgator e seu IP é: {skt.gethostbyname(site)}')
    try:
      envia_email(f'Migração de servidor do site - {site}', 'email_de_disparo', 'senha_de_app', email, msg_migracao)
      envia_email(f'Confirmação de envio - {site}', 'email_de_disparo', 'senha_de_app', 'email_recebimento', msg_enviou)
    except Exception as e:
      print(f'Erro ao tentar enviar e-mail. Erro: {e}')
  elif skt.gethostbyname(site) == 'IP2':
    print(f'O site {site} está na infonet e seu IP é: {skt.gethostbyname(site)}')
  else:
    msg_ti = f'''
        Olá!
        <br>
        <br>
        Aqui é Cássio e este e-mail está sendo gerado automaticamente.
        <br>
        <br>
        Este e-mail foi enviado pois o IP do site {site} está apontando para {skt.gethostbyname(site)} e eu gostaria de validar se de fato o cliente não está hospedado conosco.
        <br>
        <br>
        Poderia validar isso, por favor? <br><br>
        Att, <br>
        Cássio T.
      '''
    print(f'O site {site} não está hospedado com a AW e seu IP é: {skt.gethostbyname(site)}')
    try:
      envia_email(f'Cliente não está na AW - {site}', 'email_de_disparo', 'senha_de_app', 'TI@empresa.com.br', msg_ti)
    except Exception as e:
      print(f'Erro ao tentar enviar e-mail. Erro: {e}')